### Кейс 2 - определение ССЗ

1. обучить несколько разных моделей на наборе данных ССЗ (train_case2.csv): логрег, бустинг, лес и т.д - на ваш выбор 2-3 варианта
2. при обучении моделей обязательно использовать кроссвалидацию
3. вывести сравнение полученных моделей по основным метрикам классификации: pr/rec/auc/f_score (можно в виде таблицы, где строки - модели, а столбцы - метрики)
4. сделать выводы о том, какая модель справилась с задачей лучше других
5. (опциональный вопрос) какая метрика (precision_recall_curve или roc_auc_curve) больше подходит в случае сильного дисбаланса классов? (когда объектов одного из классов намного больше чем другого). 

p.s.В вопросе проще разобраться, если вспомнить оси на графике roc auc curve и рассмотреть такой пример:

Имеется 100000 объектов, из которых только 100 - класс "1" (99900 - класс "0", соответственно). 
Допустим, у нас две модели:

- первая помечает 100 объектов как класс 1, но TP = 90
- вторая помечает 1000 объектов как класс 1, но TP такой же - 90

Какая модель лучше и почему? И что позволяет легче сделать вывод - roc_auc_curve или precision_recall_curve?

ссылка на соревнование - https://mlbootcamp.ru/ru/round/12/sandbox/

В рамках конкурса вам нужно предсказать наличие сердечно-сосудистых заболеваний по результатам классического врачебного осмотра. Датасет сформирован из 100.000 реальных клинических анализов, и в нём используются признаки, которые можно разбить на 3 группы:

 

Объективные признаки:

 - Возраст
 - Рост
 - Вес
 - Пол
 

Результаты измерения:

 - Артериальное давление верхнее и нижнее
 - Холестерин
 - Глюкоза
 

Субъективные признаки:

 - Курение
 - Употребление Алкоголя
 - Физическая активность
 

Возраст дан в днях. Значения показателей холестерина и глюкозы представлены одним из трех классов: норма, выше нормы, значительно выше нормы. Значения субъективных признаков — бинарны.

Все показатели даны на момент осмотра.

Таргет - наличие сердечно-сосудистых заболеваний (ССЗ)

In [1]:
import pandas as pd
import numpy as np
# import itertools
# from tqdm.notebook import tqdm

from sklearn.metrics import precision_recall_curve, roc_curve, roc_auc_score, confusion_matrix
from sklearn.metrics import roc_auc_score, log_loss

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
%matplotlib inline

In [3]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in self.columns:
            if col_ not in test_columns:
                X[col_] = 0
        return X[self.columns]


In [4]:
DATA_FILE = 'train_case2.csv'

NUM_COLS = ['age', 'height', 'weight', 'ap_hi', 'ap_lo']
CAT_COLS = ['gender', 'cholesterol']
BASE_COLS = ['gluc', 'smoke', 'alco', 'active']


In [5]:
df = pd.read_csv('train_case2.csv', ';')
df.head(3)

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1


### Индекс Кетле
$\Large Ketle = \frac{weight}{height^2}$  

| Keltle | Category |
|--------|----------|
| 18.5   |thin      |
| 25     |normal    |
| 30     |thick     |
| 35     |fat1      |
| 40     |fat2      |
| 40+    |fat3      |

In [6]:
ketle_bins = [0, 18.5, 25, 30, 35 , 40, float('inf')]
ketle_cat = ['thin', 'normal', 'thick', 'fat1', 'fat2', 'fat3']

df['ketle'] = pd.cut(df['weight'] / (df['height'] / 100)**2, ketle_bins, right=False, labels=ketle_cat)
CAT_COLS.append('ketle')

In [7]:
num_transformers = []
cat_transformers = []
base_transformers = []

for num_col in NUM_COLS:
    transfomer =  Pipeline([
                ('selector', NumberSelector(key=num_col)),
                ('standard', StandardScaler())
            ])
    num_transformers.append((num_col, transfomer))
    
for cat_col in CAT_COLS:
    cat_transformer = Pipeline([
                ('selector', ColumnSelector(key=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    cat_transformers.append((cat_col, cat_transformer))
    
for base_col in BASE_COLS:
    base_transformer = Pipeline([
                ('selector', NumberSelector(key=base_col))
            ])
    base_transformers.append((base_col, base_transformer))
    
feats = FeatureUnion(num_transformers + cat_transformers + base_transformers)
feature_processing = Pipeline([('feats', feats)])


In [8]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns='cardio'), 
                                                    df['cardio'], test_size=0.33, random_state=42)

Теперь объединим все наши трансформеры с помощью FeatureUnion

In [9]:
feats = FeatureUnion(num_transformers + cat_transformers + base_transformers)
feature_processing = Pipeline([('feats', feats)])

feature_processing.fit_transform(X_train)

array([[ 1.01749796, -1.01586151, -0.986831  , ...,  0.        ,
         0.        ,  1.        ],
       [ 0.3334502 , -0.65065191, -0.986831  , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.38653685,  0.68844994,  1.10127359, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 1.67439454,  0.56671341, -0.01238219, ...,  0.        ,
         0.        ,  1.        ],
       [-0.49851073,  0.32324034, -0.29079614, ...,  0.        ,
         0.        ,  0.        ],
       [-1.76002771,  1.54060566, -0.70841706, ...,  0.        ,
         0.        ,  1.        ]])

Добавим классификаторы

In [10]:
log_reg_cls = Pipeline([
    ('features', feats),
    ('classifier', LogisticRegression(random_state = 42)),
])

grad_boost_cls = Pipeline([
    ('features', feats),
    ('classifier', GradientBoostingClassifier(random_state = 42)),
])

rnd_forest_cls = Pipeline([
    ('features', feats),
    ('classifier', RandomForestClassifier(random_state = 42)),
])

In [11]:
%%time
classifiers = {'log_reg': log_reg_cls, 
               'grad_boost':grad_boost_cls, 
               'rnd_forest': rnd_forest_cls}

#запустим кросс-валидацию
for name, classifier in classifiers.items():
    cv_scores = cross_val_score(classifier, X_train, y_train, cv=16, scoring='roc_auc')
    cv_score = np.mean(cv_scores)
    cv_score_std = np.std(cv_scores)
    print('{}, CV score is {}+-{}'.format(name, cv_score, cv_score_std))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


log_reg, CV score is 0.7790985146440751+-0.010689552089496132
grad_boost, CV score is 0.8004024235388529+-0.009310431725674771
rnd_forest, CV score is 0.7696749067524734+-0.011391836439990236
Wall time: 2min 19s


In [12]:
%%time
#обучим пайплайн на всем тренировочном датасете
for name, classifier in classifiers.items():
    classifier.fit(X_train, y_train)

Wall time: 9.21 s


In [13]:
y_scores = {}
for name, classifier in classifiers.items():
    y_scores[name] = classifier.predict_proba(X_test)[:, 1]
# y_score = classifier.predict_proba(X_test)[:, 1]

Посчитаем precision/recall/f_score

In [14]:
for name, y_score in y_scores.items():
    b = 1
    precision, recall, thresholds = precision_recall_curve(y_test.values, y_score)
    fscore = (1 + b**2) * (precision * recall) / (b**2 * precision + recall)
    # locate the index of the largest f score
    ix = np.argmax(fscore)
    print('For %s:\nBest Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (name,
                                                                                     thresholds[ix], 
                                                                                     fscore[ix],
                                                                                     precision[ix],
                                                                                     recall[ix]))
    print("roc auc score: {}\n".format(roc_auc_score(y_true=y_test, y_score=y_score)))

For log_reg:
Best Threshold=0.402838, F-Score=0.735, Precision=0.658, Recall=0.833
roc auc score: 0.7859353578145292

For grad_boost:
Best Threshold=0.393515, F-Score=0.747, Precision=0.703, Recall=0.797
roc auc score: 0.8057358438154729

For rnd_forest:
Best Threshold=0.360000, F-Score=0.722, Precision=0.648, Recall=0.815
roc auc score: 0.7708045747092944



Лучшие результаты показал градиентный бустинг, добавленный идекс Кетле чуть-чуть ухудшил модель